### Data Loading and Preprocessing

In [ ]:
import numpy as np
import random
import os
import tensorflow as tf


# Set a fixed seed value for reproducibility
SEED = 1
random.seed(SEED)            # Python random module
np.random.seed(SEED)         # NumPy
tf.random.set_seed(SEED)     # TensorFlow

# Enforce deterministic behavior for GPU operations
os.environ['TF_DETERMINISTIC_OPS'] = '1'  # Ensure deterministic execution
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # Deterministic cuDNN algorithms

# Control GPU memory allocation (prevents TensorFlow from using all GPU memory)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)  # Enable memory growth

# Restrict parallelism (ensures consistent execution order)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

import scipy.io 
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report, 
f1_score, precision_score, recall_score, roc_auc_score)
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import signal
from skimage.transform import resize
from sklearn.model_selection import StratifiedKFold
from keras import layers, models, Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model

# Load 2HP CWRU bearing data from .mat files
def import_data():
    """
    Efficiently load CWRU bearing data from .mat files in the specified folder.

    Returns:
        list: A list of NumPy arrays, each corresponding to a different bearing condition.
    """
    folder_path_1 = os.path.join(os.getcwd(), 'CWRU_data', '2HP')

    files = {
        '99.mat': 'X099_DE_time',
        '111.mat': 'X111_DE_time',
        '124.mat': 'X124_DE_time',
        '137.mat': 'X137_DE_time',
        '176.mat': 'X176_DE_time',
        '191.mat': 'X191_DE_time',
        '203.mat': 'X203_DE_time',
        '215.mat': 'X215_DE_time',
        '228.mat': 'X228_DE_time',
        '240.mat': 'X240_DE_time',
    }

    data_2HP = []
    for file_name, key in files.items():
        file_path = os.path.join(folder_path_1, file_name)
        data_2HP.append(scipy.io.loadmat(file_path)[key])

    folder_path_2 =  os.path.join(os.getcwd(), 'CWRU_data', '3HP')
    files = {
        '100.mat': 'X100_DE_time',
        '112.mat': 'X112_DE_time',
        '125.mat': 'X125_DE_time',
        '138.mat': 'X138_DE_time',
        '177.mat': 'X177_DE_time',
        '192.mat': 'X192_DE_time',
        '204.mat': 'X204_DE_time',
        '217.mat': 'X217_DE_time',
        '229.mat': 'X229_DE_time',
        '241.mat': 'X241_DE_time',
    }

    data_3HP = []
    for file_name, key in files.items():
        file_path = os.path.join(folder_path_2, file_name)
        data_3HP.append(scipy.io.loadmat(file_path)[key])

    return data_2HP, data_3HP

data_2HP, data_3HP = import_data()
len(data_2HP), len(data_3HP)

In [ ]:
def sampling(data, interval_length, samples_per_block, ignore_points=0):
    """
    Split the data into blocks of specified length, ignoring the first and last 'ignore_points'.
    Args:
        data (np.ndarray): The input data array to be split.
        interval_length (int): The length of each interval for splitting.
        samples_per_block (int): The number of samples in each block.
        ignore_points (int): The number of points to ignore at the beginning and end of the data.
    Returns:
        np.ndarray: A 2D array where each row is a block of data.
    """ 
    # Adjust data length to ignore the first and last 'ignore_points'
    adjusted_length = len(data) - 2 * ignore_points

    # Adjust the number of blocks
    no_of_blocks = (round(adjusted_length / interval_length) - round(samples_per_block / interval_length) - 1)
    split_data = np.zeros([no_of_blocks, samples_per_block])

    for i in range(no_of_blocks):
        # Skip the first 'ignore_points' and start sampling from that position
        start_idx = ignore_points + i * interval_length
        split_data[i, :] = data[start_idx:(start_idx + samples_per_block)].T

        # print("Split data shape", split_data[i, :].shape)

    return split_data


def data_preparation(data, interval_length, samples_per_block):
  for count,i in enumerate(data):
    split_data = sampling(i, interval_length, samples_per_block)
    y = np.zeros([len(split_data),10])
    y[:,count] = 1
    y1 = np.zeros([len(split_data),1])
    y1[:,0] = count
    # Stack up and label the data   
    if count==0:
      X = split_data
      y_positional = y
      y_labels = y1
    else:
      X = np.append(X, split_data, axis=0)
      y_positional = np.append(y_positional,y,axis=0)
      y_labels = np.append(y_labels,y1,axis=0)
  return X, y_positional, y_labels

### Overlapping Windows

In [ ]:
interval_length = 320 
samples_per_block = 1600

'''Y_CNN is of shape (n, 10) representing 10 classes as 10 columns. In each sample, for the class to which it belongs, 
the corresponding column value is marked 1 and the rest as 0, facilitating Softmax implementation in CNN 
Y is of shape (m, 1) where column values are between 0 and 9 representing the classes directly, 1-hot encoding'''

X_2HP, y_onehot_2HP, y_2HP = data_preparation(data_2HP, interval_length, samples_per_block) 
X_3HP, y_onehot_3HP, y_3HP = data_preparation(data_3HP, interval_length, samples_per_block)

print('Shape of input data (2HP) =', X_2HP.shape)
print('Shape of one hot encoded label(2HP)  =', y_onehot_2HP.shape)
print('Shape of labels (2HP) =', y_2HP.shape)

print('Shape of input data (3HP) =', X_3HP.shape)
print('Shape of one hot encoded label(3HP)  =', y_onehot_3HP.shape)
print('Shape of labels (3HP) =', y_3HP.shape)


### Time-Based Train-Test Split

In [ ]:
def time_series_stratified_split(X, y, train_ratio = 0.8):
    num_classes = y.shape[1]
    X_train, y_train, X_test, y_test = [], [], [], []

    for cls in range(num_classes):
        cls_indices = np.where(np.argmax(y, axis=1) == cls)[0]
        n_train = int(train_ratio * len(cls_indices))
        train_idx, test_idx = cls_indices[:n_train], cls_indices[n_train:]
        X_train.append(X[train_idx])
        # print("X_train shape:", len(X_train))
        y_train.append(y[train_idx])
        # print("y_train shape:", len(X_train))
        X_test.append(X[test_idx])
        # print("X_train shape:", len(X_train))
        y_test.append(y[test_idx])
        # print("y_test shape:", len(X_train))
        print(f"Class {cls}: Train size = {len(train_idx)}, Test size = {len(test_idx)}")

    return (
        np.concatenate(X_train),
        np.concatenate(y_train),
        np.concatenate(X_test),
        np.concatenate(y_test)
    )

X_train_2HP, y_train_2HP, X_test_2HP, y_test_2HP = time_series_stratified_split(X_2HP, y_onehot_2HP)
X_train_3HP, y_train_3HP, X_test_3HP, y_test_3HP = time_series_stratified_split(X_3HP, y_onehot_3HP)

print("Shape of training data (2HP):", X_train_2HP.shape)
print("Shape of training labels (2HP):", y_train_2HP.shape)
print("Shape of testing data (2HP):", X_test_2HP.shape)
print("Shape of testing labels (2HP):", y_test_2HP.shape)

In [ ]:
# k-fold cross validation 
k_splits = 5
kfold = StratifiedKFold(n_splits=k_splits, shuffle=False)

### Baseline Model Definition

#### 2D CNN

In [ ]:
class CNN_2D():
  def __init__(self):
    self.model = self.CreateModel()

  def CreateModel(self):
    model = models.Sequential([
        layers.Conv2D(filters=16, kernel_size=(3,3), padding='same',activation='relu', input_shape=(40,40,1)),
        layers.MaxPool2D(pool_size=(2,2), padding='same'),

        layers.Conv2D(filters=32, kernel_size=(3,3), padding ='same',activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), padding='same'),

        layers.Conv2D(filters=64, kernel_size=(3,3),padding ='same', activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), padding='same'),

        layers.Conv2D(filters=128, kernel_size=(3,3),padding ='same', activation='relu'),
        layers.MaxPool2D(pool_size=(2,2), padding='same'),
        layers.Flatten(),
        
        layers.Dense(100,activation='relu'),
        layers.Dense(50,activation='relu'),
        layers.Dense(10),
        layers.Softmax()
        ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
    return model

#### 1D CNN

In [ ]:
class CNN_1D():
    def __init__(self):
        self.model = self.CreateModel()
        self.model.summary()

    def CreateModel(self):
        model = models.Sequential([
            layers.Conv1D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=input_shape),
            layers.BatchNormalization(),
            layers.MaxPool1D(pool_size=2),
            
            layers.Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPool1D(pool_size=2),

            layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPool1D(pool_size=2),
            layers.GlobalAveragePooling1D(),
            
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(10, activation='softmax')
        ])

        # Optimizer with a slightly higher learning rate
        model.compile(optimizer='adam',
                      loss=tf.keras.losses.CategoricalCrossentropy(),
                      metrics=['accuracy'])
        return model

In [ ]:
# Define folder path and reshape the data for the models
# File path name to save best models
foldername = "CNN2D_results_for_report/Baseline/"
os.makedirs(foldername, exist_ok=True)

# Reshape the data for 2D CNN input
X_train_2HP_2D = X_train_2HP.reshape(X_train_2HP.shape[0], 40, 40, 1)
X_train_3HP_2D = X_train_3HP.reshape(X_train_3HP.shape[0], 40, 40, 1)
y_train_2HP_2D = y_train_2HP.reshape(y_train_2HP.shape[0], 10)
y_train_3HP_2D = y_train_3HP.reshape(y_train_3HP.shape[0], 10)


X_test_2HP_2D = X_test_2HP.reshape(X_test_2HP.shape[0], 40, 40, 1)
X_test_3HP_2D = X_test_3HP.reshape(X_test_3HP.shape[0], 40, 40, 1)  
y_test_2HP_2D = y_test_2HP.reshape(y_test_2HP.shape[0], 10)
y_test_3HP_2D = y_test_3HP.reshape(y_test_3HP.shape[0], 10)

# Reshape the data for 1D CNN input
X_train_2HP_1D = X_train_2HP.reshape(X_train_2HP.shape[0], 40, 40)
X_train_3HP_1D = X_train_3HP.reshape(X_train_3HP.shape[0], 40, 40)
y_train_2HP_1D = y_train_2HP.reshape(y_train_2HP.shape[0], 10)
y_train_3HP_1D = y_train_3HP.reshape(y_train_3HP.shape[0], 10)

X_test_2HP_1D = X_test_2HP.reshape(X_test_2HP.shape[0], 40, 40)
X_test_3HP_1D = X_test_3HP.reshape(X_test_3HP.shape[0], 40, 40)
y_test_2HP_1D = y_test_2HP.reshape(y_test_2HP.shape[0], 10)
y_test_3HP_1D = y_test_3HP.reshape(y_test_3HP.shape[0], 10)

### Experiment Loop Function

In [ ]:
import json
from datetime import datetime
def save_metrics(metrics, filename, fold, model_type, hp):
    """
    Save model metrics to a JSON file.
    """
    metrics_dir = os.path.join(foldername, "CNN2D_results_for_report", "Metrics", model_type, hp)
    os.makedirs(metrics_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(metrics_dir, f"{filename}_fold{fold}_{timestamp}.json")
    with open(filepath, 'w') as f:
        json.dump(metrics, f, indent=4)

def save_best_model(model, fold, model_type, hp, val_acc):
    """
    Save the best model for a given fold.
    """
    models_dir = os.path.join(foldername, "CNN2D_results_for_report", "Models", model_type, hp)
    os.makedirs(models_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(models_dir, f"best_model_fold{fold}_val_acc_{val_acc:.4f}_{timestamp}.h5")
    model.save(filepath)

def plot_confusion_matrix(y_true, y_pred, fold, model_type, hp, class_names):
    """
    Plot and save confusion matrix.
    """
    cm = confusion_matrix(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_type} ({hp}) Fold {fold}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

    plots_dir = os.path.join(foldername, "CNN2D_results_for_report", "Plots", model_type, hp)
    os.makedirs(plots_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(plots_dir, f"confusion_matrix_fold{fold}_{timestamp}.png")
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

def plot_training_curves(history, fold, model_type, hp):
    """
    Plot and save training/validation accuracy and loss curves.
    """
    plots_dir = os.path.join(foldername, "CNN2D_results_for_report", "Plots", model_type, hp)
    os.makedirs(plots_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy Curves - {model_type} ({hp}) Fold {fold}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    filepath = os.path.join(plots_dir, f"accuracy_curve_fold{fold}_{timestamp}.png")
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()
    
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves - {model_type} ({hp}) Fold {fold}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    filepath = os.path.join(plots_dir, f"loss_curve_fold{fold}_{timestamp}.png")
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

def run_experiment(X_train, y_train, X_test, y_test, model_class, kfold, model_type, hp, class_names):
    """
    Run k-fold cross-validation experiment and save results.
    """
    fold_metrics = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, np.argmax(y_train, axis=1))):
        print(f"\nTraining Fold {fold + 1}/{kfold.n_splits}")
        
        model = model_class().model
        checkpoint = ModelCheckpoint(
            f"temp_best_model_fold{fold}.h5",
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )
        early_stopping = EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True
        )
        
        history = model.fit(
            X_train[train_idx], y_train[train_idx],
            validation_data=(X_train[val_idx], y_train[val_idx]),
            epochs=50,
            batch_size=32,
            callbacks=[checkpoint, early_stopping],
            verbose=1
        )

        best_model = tf.keras.models.load_model(f"temp_best_model_fold{fold}.h5") # Load the best model

        train_pred = best_model.predict(X_train[train_idx])
        val_pred = best_model.predict(X_train[val_idx])
        test_pred = best_model.predict(X_test)
        
        train_acc = tf.keras.metrics.categorical_accuracy(y_train[train_idx], train_pred).numpy().mean()
        val_acc = tf.keras.metrics.categorical_accuracy(y_train[val_idx], val_pred).numpy().mean()
        test_acc = tf.keras.metrics.categorical_accuracy(y_test, test_pred).numpy().mean()
        
        report = classification_report(
            np.argmax(y_test, axis=1),
            np.argmax(test_pred, axis=1),
            target_names=class_names,
            output_dict=True
        )
        
        metrics = {
            'fold': fold + 1,
            'train_accuracy': float(train_acc),
            'val_accuracy': float(val_acc),
            'test_accuracy': float(test_acc),
            'classification_report': report
        }
        save_metrics(metrics, "metrics", fold + 1, model_type, hp)
        save_best_model(best_model, fold + 1, model_type, hp, val_acc)
        plot_confusion_matrix(y_test, test_pred, fold + 1, model_type, hp, class_names)
        plot_training_curves(history, fold + 1, model_type, hp)
        
        fold_metrics.append(metrics)
        os.remove(f"temp_best_model_fold{fold}.h5")
    
    avg_metrics = {
        'avg_train_accuracy': float(np.mean([m['train_accuracy'] for m in fold_metrics])),
        'avg_val_accuracy': float(np.mean([m['val_accuracy'] for m in fold_metrics])),
        'avg_test_accuracy': float(np.mean([m['test_accuracy'] for m in fold_metrics]))
    }
    save_metrics(avg_metrics, "average_metrics", 0, model_type, hp)
    
    return fold_metrics

### Run 4 Experiments

In [ ]:
class_names = [
    'Normal', 'Inner Race 0.007"', 'Inner Race 0.014"', 'Inner Race 0.021"',
    'Ball 0.007"', 'Ball 0.014"', 'Ball 0.021"',
    'Outer Race 0.007"', 'Outer Race 0.014"', 'Outer Race 0.021"'
]

# Run experiments
print("Running 2HP CNN_2D Experiment")
metrics_2hp_cnn2d = run_experiment(
    X_train_2HP_2D, y_train_2HP_2D, X_test_2HP_2D, y_test_2HP_2D,
    CNN_2D, kfold, "CNN_2D", "2HP", class_names
)

print("\nRunning 2HP CNN_1D Experiment")
metrics_2hp_cnn1d = run_experiment(
    X_train_2HP_1D, y_train_2HP_1D, X_test_2HP_1D, y_test_2HP_1D,
    CNN_1D, kfold, "CNN_1D", "2HP", class_names
)

print("\nRunning 3HP CNN_2D Experiment")
metrics_3hp_cnn2d = run_experiment(
    X_train_3HP_2D, y_train_3HP_2D, X_test_3HP_2D, y_test_3HP_2D,
    CNN_2D, kfold, "CNN_2D", "3HP", class_names
)

print("\nRunning 3HP CNN_1D Experiment")
metrics_3hp_cnn1d = run_experiment(
    X_train_3HP_1D, y_train_3HP_1D, X_test_3HP_1D, y_test_3HP_1D,
    CNN_1D, kfold, "CNN_1D", "3HP", class_names
)

In [ ]:
# Example of loading and displaying metrics
import glob
import pandas as pd

def load_latest_metrics(metrics_dir, filename_pattern):
    """Load the latest JSON file matching the pattern in the given directory."""
    files = glob.glob(os.path.join(metrics_dir, f"{filename_pattern}*.json"))
    if not files:
        print(f"No files found in {metrics_dir} for pattern {filename_pattern}")
        return None
    latest_file = max(files, key=os.path.getctime)  # Get the most recent file
    with open(latest_file, 'r') as file:
        return json.load(file)

# Load average metrics for all experiments
experiments = [
    ("CNN_1D", "2HP", "average_metrics_fold0"),
    ("CNN_1D", "3HP", "average_metrics_fold0"),
    ("CNN_2D", "2HP", "average_metrics_fold0"),
    ("CNN_2D", "3HP", "average_metrics_fold0")
]

for model_type, hp, pattern in experiments:
    metrics_dir = os.path.join("CNN2D_results_for_report", "Metrics", model_type, hp)
    data = load_latest_metrics(metrics_dir, pattern)
    if data:
        df = pd.DataFrame(data, index=[0])
        print(f"\nAverage Metrics for {model_type} ({hp}):")
        print(df)

# Load and display per-class classification report for a specific fold
metrics_dir = os.path.join("CNN2D_results_for_report", "Metrics", "CNN_2D", "2HP")
data = load_latest_metrics(metrics_dir, "metrics_fold1")
if data:
    print("\nPer-Class Classification Report (CNN_2D, 2HP, Fold 1):")
    print(f"Fold: {data['fold']}")
    print(f"Train Accuracy: {data['train_accuracy']:.4f}")
    print(f"Validation Accuracy: {data['val_accuracy']:.4f}")
    print(f"Test Accuracy: {data['test_accuracy']:.4f}")
    
    report = data["classification_report"]
    class_report = {k: v for k, v in report.items() if isinstance(v, dict)}
    df_report = pd.DataFrame(class_report).T
    print("\nPer-Class Metrics:")
    print(df_report)

In [ ]:
# Summary table of average metrics
summary_data = []
for model_type, hp, pattern in experiments:
    metrics_dir = os.path.join("CNN2D_results_for_report", "Metrics", model_type, hp)
    data = load_latest_metrics(metrics_dir, pattern)
    if data:
        summary_data.append({
            'Model': model_type,
            'HP': hp,
            'Avg Train Accuracy': data['avg_train_accuracy'],
            'Avg Val Accuracy': data['avg_val_accuracy'],
            'Avg Test Accuracy': data['avg_test_accuracy']
        })

summary_df = pd.DataFrame(summary_data)
print("\nSummary of Average Metrics Across Experiments:")
print(summary_df)

In [ ]:
# Aggregate per-class metrics across folds for CNN_2D (2HP)
metrics_dir = os.path.join("CNN2D_results_for_report", "Metrics", "CNN_2D", "2HP")
fold_files = glob.glob(os.path.join(metrics_dir, "metrics_fold*.json"))

class_metrics = []
for fold_file in fold_files:
    with open(fold_file, 'r') as f:
        data = json.load(f)
        report = data['classification_report']
        for cls in class_names:
            class_metrics.append({
                'Fold': data['fold'],
                'Class': cls,
                'Precision': report[cls]['precision'],
                'Recall': report[cls]['recall'],
                'F1-Score': report[cls]['f1-score']
            })

class_df = pd.DataFrame(class_metrics)
avg_class_metrics = class_df.groupby('Class').mean().reset_index()
print("\nAverage Per-Class Metrics for CNN_2D (2HP):")
print(avg_class_metrics[['Class', 'Precision', 'Recall', 'F1-Score']])

In [ ]:
# Bar chart comparing average test accuracies
summary_data = []
for model_type, hp, pattern in experiments:
    metrics_dir = os.path.join("CNN2D_results_for_report", "Metrics", model_type, hp)
    data = load_latest_metrics(metrics_dir, pattern)
    if data:
        summary_data.append({
            'Model': f"{model_type} ({hp})",
            'Avg Test Accuracy': data['avg_test_accuracy']
        })

summary_df = pd.DataFrame(summary_data)

In [ ]:
def load_latest_metrics(metrics_dir, filename_pattern):
    files = glob.glob(os.path.join(metrics_dir, f"{filename_pattern}*.json"))
    if not files:
        print(f"No files found in {metrics_dir} for pattern {filename_pattern}")
        return None
    latest_file = max(files, key=os.path.getctime)
    try:
        with open(latest_file, 'r') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading {latest_file}: {e}")
        return None

In [ ]:
# # check the saved metrics for 3HP CNN_1D
# import pandas as pd
# import json

# with open('CNN2D_results_for_report/Metrics/CNN_1D/2HP/average_metrics_fold0_20250707_144506.json', 'r') as file:
#     data = json.load(file)

# metrics_2hp_cnn1d = pd.DataFrame(data, index=[0])
# metrics_2hp_cnn1d

In [ ]:
# import json
# import pandas as pd

# # Path to your JSON file
# json_path = "/Users/user/Desktop/MEng_UWaterloo/7_ECE_699A_PRJ/Code_Final_Version/CNN2D_results_for_report/Metrics/CNN_2D/2HP/metrics_fold1_20250707_141802.json"

# # Load JSON data
# with open(json_path, 'r') as f:
#     data = json.load(f)

# # Print basic metrics
# print("Fold:", data["fold"])
# print("Train Accuracy:", data["train_accuracy"])
# print("Validation Accuracy:", data["val_accuracy"])
# print("Test Accuracy:", data["test_accuracy"])

# # Extract and convert classification report to DataFrame (excluding overall/macro/weighted)
# report = data["classification_report"]

# # Filter out entries that are not class names
# class_report = {k: v for k, v in report.items() if isinstance(v, dict)}

# # Convert to DataFrame
# df_report = pd.DataFrame(class_report).T  # Transpose for readability

# # Display the DataFrame
# print("\nPer-Class Classification Report:")
# print(df_report)
